## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-09-15-45-39 +0000,nyt,Trump Administration Demands States ‘Undo’ Wor...,https://www.nytimes.com/2025/11/09/business/tr...
1,2025-11-09-15-33-10 +0000,nypost,Lefty Gen-Z NYC Councilman Chi Osse gears up f...,https://nypost.com/2025/11/09/us-news/nyc-coun...
2,2025-11-09-15-30-50 +0000,startribune,MSP flight cancellations climb through Sunday ...,https://www.startribune.com/msp-flight-cancell...
3,2025-11-09-15-28-57 +0000,nypost,Parents issue dire travel warning after daught...,https://nypost.com/2025/11/09/world-news/paren...
4,2025-11-09-15-27-44 +0000,nyt,Blue Origin to Launch NASA’s ESCAPADE Mission ...,https://www.nytimes.com/2025/11/09/science/blu...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,21
16,up,8
148,new,8
89,year,7
107,president,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
161,2025-11-08-17-30-00 +0000,wsj,"Middle-class voters, many of them frustrated w...",https://www.wsj.com/politics/elections/democra...,60
109,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...,60
108,2025-11-09-02-00-00 +0000,wsj,Trump’s Tax Cuts Are Exposing Companies to Bid...,https://www.wsj.com/politics/policy/trump-bide...,42
168,2025-11-08-17-17-26 +0000,nypost,New Yorkers flock to food banks as government ...,https://nypost.com/2025/11/08/us-news/trump-ur...,42
147,2025-11-08-20-13-16 +0000,nypost,US lifts sanctions on Syrian president ahead o...,https://nypost.com/2025/11/08/world-news/us-li...,41


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
161,60,2025-11-08-17-30-00 +0000,wsj,"Middle-class voters, many of them frustrated w...",https://www.wsj.com/politics/elections/democra...
168,42,2025-11-08-17-17-26 +0000,nypost,New Yorkers flock to food banks as government ...,https://nypost.com/2025/11/08/us-news/trump-ur...
109,39,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...
163,32,2025-11-08-17-25-55 +0000,nypost,Mamdani-backing Hochul insists she doesn’t ‘su...,https://nypost.com/2025/11/08/us-news/mamdani-...
19,26,2025-11-09-14-06-33 +0000,nypost,Elon Musk trolled mercilessly after posting AI...,https://nypost.com/2025/11/09/us-news/elon-mus...
57,24,2025-11-09-10-26-24 +0000,cbc,Super Typhoon Fung-wong hits Philippines after...,https://www.cbc.ca/news/world/super-typhoon-th...
164,24,2025-11-08-17-23-43 +0000,nypost,"Andrew Mountbatten Windsor, Fergie being probe...",https://nypost.com/2025/11/08/world-news/princ...
115,23,2025-11-09-00-47-19 +0000,nypost,Air Force families at Florida base ordered to ...,https://nypost.com/2025/11/08/us-news/air-forc...
175,22,2025-11-08-16-05-23 +0000,nyt,At Least 4 Dead After Driver Slams Into Crowd ...,https://www.nytimes.com/2025/11/08/us/car-cras...
139,19,2025-11-08-21-54-38 +0000,nypost,Beware the ‘high’ seas: Border agents doing ‘m...,https://nypost.com/2025/11/08/us-news/border-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
